# Logistic Regression

In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Simulierte Daten

In [2]:
def generate_prices(observations_count, prices_range):
    return np.random.choice(prices_range, observations_count)

def generate_competitor_prices(observations_count, competitors_count, prices_range):
    return np.matrix([generate_prices(observations_count, prices_range) for i in range(0, competitors_count)])
    
def calculate_ranks(observations_count, competitors_count, competitor_prices, prices):
    return [1 + len([1 for j in range(competitors_count) if competitor_prices[j,i] < prices[i]])
            for i in range(observations_count)]

def calculate_sold_probs_A(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[0][i])/(competitors_count[0] + 1)) - 0.05 * prices[0][i] + (-0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

def calculate_sold_probs_B(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[1][i])/(competitors_count[1] + 1)) - 0.05 * prices[1][i] + (0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

observations_count = 1000
prices_ranges = [np.arange(1, 20.1, 0.1) for i in range(2)] #TODO adopt prob function if price changes
prices = [generate_prices(observations_count, prices_ranges[i]) for i in range(2)] # TODO p1 > p2?
competitors_count = [5, 5]
competitor_prices = [generate_competitor_prices(observations_count, competitors_count[i], prices_ranges[i]) for i in range(2)]
ranks = [calculate_ranks(observations_count, competitors_count[i], competitor_prices[i], prices[i]) for i in range(2)]
sold = [calculate_sold_probs_A(observations_count, ranks, competitors_count, prices),
        calculate_sold_probs_B(observations_count, ranks, competitors_count, prices)]

## Regression

In [3]:
def get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[0][j] for j in range(observations_count)]
    explanatory_3 = [prices[0][j] - np.min([competitor_prices[0][k,j] for k in range(0, competitors_count[0])]) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3])

def get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[1][j] for j in range(observations_count)]
    explanatory_3 = [prices[1][j] - np.min([competitor_prices[1][k,j] for k in range(0, competitors_count[1])]) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3])

explanatory_vars = [get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices),
                    get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices)]
logits = [sm.Logit(sold[i], explanatory_vars[i].transpose()) for i in range(2)]
results = [logits[i].fit() for i in range(2)]
beta = [results[i].params for i in range(2)]
beta

Optimization terminated successfully.
         Current function value: 0.373144
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.372541
         Iterations 7


[array([ 0.33549798, -0.32205495, -0.19359509]),
 array([ 0.45311463, -0.35291189, -0.18994164])]

In [4]:
[results[i].aic for i in range(2)]

[752.2887147929921, 751.08238236782631]

## Optimierung

In [5]:
def get_price_index(price):
    return int(price / 0.1 - 10) # TODO dependent on price range?

def calculate_sale_probs(beta, explanatory_vars, prices_range):
    L = lambda price: np.sum([beta[m] * explanatory_vars[m,get_price_index(price)] for m in range(explanatory_vars.shape[0])])
    return [np.exp(L(price)) / (1 + np.exp(L(price))) for price in prices_range]

def bellman(n, sale_probs, prices_range, holding_cost_rate, delta, values, step):
    prob_A = lambda price, i: sale_probs[0, i, get_price_index(price)]
    prob_B = lambda price, j: sale_probs[1, j, get_price_index(price)]
    todays_profit = lambda prices, j, i: np.min([n[j], i]) * prices[j] - n[j] * holding_cost_rate[j]
    disc_exp_fut_profits = lambda i, j: delta * values[step + 1, np.max([0, n[0]-i]), np.max([0, n[1]-j])]

    bellman_for_combinations = []
    for price_A in prices_range[0]:
        for price_B in prices_range[1]:
            bellman_for_combinations.append(np.sum([prob_A(price_A, i) * np.sum([prob_B(price_B, j) * ((todays_profit([price_A, price_B], 0, i)+todays_profit([price_A, price_B], 1, j)) + disc_exp_fut_profits(i,j)) for j in range(2)]) for i in range(2)]))
    return np.max(bellman_for_combinations)

def bellman_check(n, sale_probs, prices, holding_cost_rate, delta, values, step):
    prob_A = lambda price, i: sale_probs[0, i, get_price_index(price)]
    prob_B = lambda price, j: sale_probs[1, j, get_price_index(price)]
    todays_profit = lambda prices, j, i: np.min([n[j], i]) * prices[j] - n[j] * holding_cost_rate[j]
    disc_exp_fut_profits = lambda i, j: delta * values[step + 1, np.max([0, n[0]-i]), np.max([0, n[1]-j])]

    return np.sum([prob_A(price_A, i) * np.sum([prob_B(price_B, j) * ((todays_profit([price_A, price_B], 0, i)+todays_profit([price_A, price_B], 1, j)) + disc_exp_fut_profits(i,j)) for j in range(2)]) for i in range(2)])

In [6]:
competitor_prices = [generate_competitor_prices(observations_count, competitors_count[i], prices_ranges[i]) for i in range(2)]
explanatory_vars = [get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices),
                    get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices)]

p = [calculate_sale_probs(beta[i], explanatory_vars[i], prices_ranges[i]) for i in range(2)]
sale_probs = np.array([[[1 - j for j in p[i]], p[i]] for i in range(2)])

delta = 0.99
holding_cost_rate = [0.01, 0.01]
init_inventory = [2, 2]
steps = 3
values = np.empty(shape=(steps + 1, init_inventory[0] + 1, init_inventory[1] + 1))
for step in range(steps, -1, -1):
    for n_A in range(init_inventory[0] + 1):
        for n_B in range(init_inventory[1] + 1):
            if step == steps or n_A == 0 or n_B == 0:
                values[step, n_A, n_B] = 0
            else:
                values[step, n_A, n_B] = bellman([n_A, n_B], sale_probs, prices_ranges, holding_cost_rate, delta, values, step)

opt_prices = np.zeros(shape=(init_inventory[0] + 1, init_inventory[1] + 1, 2))
for n_A in range(1, init_inventory[0] + 1):
    for n_B in range(1, init_inventory[1] + 1):
        for price_A in prices_ranges[0]:
            for price_B in prices_ranges[1]:
                if bellman_check([n_A, n_B], sale_probs, [price_A, price_B], holding_cost_rate, delta, values, step) == values[0, n_A, n_B]:
                    opt_prices[n_A, n_B, 0] = price_A
                    opt_prices[n_A, n_B, 1] = price_B
for n_A in range(init_inventory[0] + 1):
    for n_B in range(init_inventory[1] + 1):
        print 'n_A: ' + str(n_A) + ',n_B: ' + str(n_B) + ' - A: ' + str(opt_prices[n_A, n_B, 0]) + ',B: ' + str(opt_prices[n_A, n_B, 1])

n_A: 0,n_B: 0 - A: 0.0,B: 0.0
n_A: 0,n_B: 1 - A: 0.0,B: 0.0
n_A: 0,n_B: 2 - A: 0.0,B: 0.0
n_A: 1,n_B: 0 - A: 0.0,B: 0.0
n_A: 1,n_B: 1 - A: 17.0,B: 17.1
n_A: 1,n_B: 2 - A: 15.0,B: 19.4
n_A: 2,n_B: 0 - A: 0.0,B: 0.0
n_A: 2,n_B: 1 - A: 19.9,B: 11.5
n_A: 2,n_B: 2 - A: 19.9,B: 19.4
